In [15]:
import pandas as pd

In [16]:
df = pd.read_csv("../project_data_pipeline/data_set/world_marathon_majors.csv", encoding='ISO-8859-1', sep=';')

In [17]:
def ajusting_data(data_frame):
    df = data_frame.dropna(axis=1, how='all')
    df['date'] = data_frame['time'].str.extract(r'(\d{2}/\d{2}/\d{4})')
    df['time_race'] = df['time'].str.extract(r'(\d{2}:\d{2})')

    return df

df_clean = ajusting_data(df)
df_clean.sample()

C:\Users\anafl\AppData\Local\Temp\ipykernel_30496\2375059643.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = data_frame['time'].str.extract(r'(\d{2}/\d{2}/\d{4})')


,year,winner,gender,country,time,marathon,date,time_race
362,1980,Alberto Salazar,Male,United States,04/05/2018 02:09,NYC,04/05/2018,02:09


In [18]:
def minutes(time):
    time_str = str(time)
    parts = time_str.split(':')
    if len(parts) == 2:
        hours, minutes = map(int, parts)
        return hours * 60 + minutes
    else:
        return 0

# Apply the conversion function to the 'Race Time' column and create a new column 'Time in Minutes'
df_clean['minutes'] = df_clean['time_race'].apply(minutes)
df_clean.sample()

,year,winner,gender,country,time,marathon,date,time_race,minutes
337,1993,Uta Pippig,Female,Germany,04/05/2018 02:26,NYC,04/05/2018,02:26,146


In [24]:
# Converting in CSV 

def converting (data_frame):
    data_frame.to_csv('../project_data_pipeline/data_set/data_set_clean')
    return "converted in a data fram"

new_df = converting(df_clean)

In [26]:
hdi = pd.read_csv(f"../project_data_pipeline/data_set/hdi_data_set")
hdi.sample()

,Unnamed: 0,Nation,HDI
104,104,Paraguay,0.717


In [28]:
# Calculate the total number of winners for each country

def grouping_winners(data_frame):
    # Creating a subset with the nationality and the total of winners
    total_winners = data_frame['country'].value_counts().reset_index()
    total_winners.columns = ['Country', 'Total Winners']
    # Getting the percentage
    total_winners['Percentage'] = ((total_winners['Total Winners'] / total_winners['Total Winners'].sum()) * 100).round(2).astype(str) + '%'
    return total_winners

total_winners = grouping_winners(df)
total_winners.head()


,Country,Total Winners,Percentage
0,Kenya,136,25.37%
1,United States,104,19.4%
2,Ethiopia,51,9.51%
3,Germany,36,6.72%
4,United Kingdom,35,6.53%


In [22]:
# Creating a function that will be responsible for combining the two data frame 

def combining_data(data_set1, data_set2, column1, column2):
    # Combininf data_set1 with data_set2 on the column1 and column2
    combined = data_set1.merge(data_set2, left_on=f'{column1}', right_on=f'{column2}')
    combined = combined[['Country', 'Total Winners', 'Percentage', 'HDI']]
    return combined

combined = combining_data(total_winners, hdi, "Country", "Nation")
combined.head()

,Country,Total Winners,Percentage,HDI
0,Kenya,136,25.37%,0.575
1,United States,104,19.4%,0.921
2,Ethiopia,51,9.51%,0.498
3,Germany,36,6.72%,0.942
4,United Kingdom,35,6.53%,0.929


In [30]:
combined.to_csv(f'../project_data_pipeline/data_set/combined_data_set')